<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#4.-Inference" data-toc-modified-id="4.-Inference-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>4. Inference</a></span><ul class="toc-item"><li><span><a href="#4-1.-(just-for-test)-Data-Preparation" data-toc-modified-id="4-1.-(just-for-test)-Data-Preparation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>4-1. (just for test) Data Preparation</a></span></li></ul></li></ul></div>

# 4. Inference

In [1]:
import sys, os
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")
import pandas as pd
import numpy as np
import torch

from Clust.clust.transformation.type.DFToNPArray import transDFtoNP, trans_df_to_np, trans_df_to_np_inf
from Clust.clust.ML.tool import data as ml_data
from Clust.clust.ML.tool import scaler as ml_scaler
from Clust.clust.ML.tool import clean as ml_clean

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as influx_Client
from Clust.clust.ingestion.mongo.mongo_client import MongoClient

db_client = influx_Client.InfluxClient(ins.CLUSTDataServer2)
mongo_client = MongoClient(ins.CLUSTMetaInfo2)

#import main_regression as mr
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cpu is available.


## 4-1. (just for test) Data Preparation 

In [2]:
# get integrated data name
bucket_name = 'integration'

# scaler path
scalerPath = './scaler/'

In [ ]:
app_name= "energy" # "Hs2SwineFarmWithWeatherTime", "energy"
model_method = 'GRU_rg'
model_clean = True
step = 'test'
bucket_name = 'integration'
# data_clean_level=0

In [3]:
db_name = 'model'
collection_name = 'meta'

search = {'modelName': 'classification_actionPattern_LSTM_cf_cleanLevel0'}

model_meta= mongo_client.get_document_by_json(db_name, collection_name, search)[0]
model_meta

IndexError: list index out of range

In [ ]:
model_name = 'energy_GRU_rg_False'

collection_name = 'meta'
db_name = 'model'
model_meta_list= mongo_client.get_document_by_json(db_name, collection_name, {'modelName': model_name})
model_meta = model_meta_list[0] ## ??

In [ ]:
collection_list = mongo_client.get_collection_list(bucket_name)
collection_list

In [ ]:
# 1. Select Test Data
window_num = 128

data_name_X = 'classification_actionPattern_testX_cleanLevel0'
dataX = db_client.get_data(bucket_name, data_name_X)[:window_num]

clean_level = model_meta["trainDataInfo"]["clean_level"]
integration_freq_sec = model_meta["trainDataInfo"]["integration_param"]["integration_frequency"]

In [ ]:
feature_list = model_meta["featureList"]
target = model_meta["target"]
model_method = model_meta["model_method"]
scaler_param = model_meta["scalerParam"]
model_file_path = model_meta['files']['modelFile']["filePath"]

X_scaler_file_path = model_meta['files']['XScalerFile']["filePath"]
y_scaler_file_path = model_meta['files']['yScalerFile']["filePath"]

infer_X, scaler_X = ml_scaler.get_scaled_test_data(dataX[feature_list], X_scaler_file_path, scaler_param)
if scaler_param == 'scale':
    scaler_y = ml_scaler.get_scaler_file(y_scaler_file_path)


dim = None

if model_method == "FC_cf":
    dim = 2

In [ ]:
# transformParameter = model_meta['transformParameter']
# window_num = transformParameter['window_num']
inferX = trans_df_to_np_inf(dataX, window_num, dim)
# inferX = np.array(inferX)

In [ ]:
modelParameter = model_meta["modelParameter"]

inferParameter = {
    'device': device,
    'batch_size': 1,
    'scaler_param': "noScale"
}

In [ ]:
device

In [ ]:
# 4. Inference
from Clust.clust.ML.classification.inference import ClassificationInference as CI

ci = CI()
ci.set_param(inferParameter)
ci.set_model(model_method, model_file_path, modelParameter)
ci.set_data(inferX)
preds = ci.inference()
print(preds)

In [ ]:
if scaler_param =='scale':
    base_df_for_inverse = pd.DataFrame(columns=target, index=range(len(preds)))
    base_df_for_inverse[target] = preds
    prediction_result = pd.DataFrame(scaler_X.inverse_transform(base_df_for_inverse), columns=target, index=base_df_for_inverse.index)
else:
    prediction_result = pd.DataFrame(data={'value':preds}, index=range(len(preds)))

In [ ]:
prediction_result